<a href="https://colab.research.google.com/github/wsanjay/Interesting_notebooks_collection/blob/main/Unstructured_io_%2B_LlamaIndex_Llama_Pack_for_Complex_PDF_Retrieval_Step_by_Step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Under the Hood

In [ ]:
!pip install llama-index llama-hub unstructured==0.10.18 lxml cohere -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 929.3/929.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 67.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.3 

In [ ]:
from pydantic import BaseModel
from unstructured.partition.html import partition_html
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key: ")

OpenAI API Key: ··········


In [ ]:
!sudo apt install "./pdf2htmlEX-0.18.8.rc1-master-20200630-Ubuntu-bionic-x86_64.deb" -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'pdf2htmlex' instead of './pdf2htmlEX-0.18.8.rc1-master-20200630-Ubuntu-bionic-x86_64.deb'
The following NEW packages will be installed:
  pdf2htmlex
0 upgraded, 1 newly installed, 0 to remove and 15 not upgraded.
Need to get 0 B/4,562 kB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 /content/pdf2htmlEX-0.18.8.rc1-master-20200630-Ubuntu-bionic-x86_64.deb pdf2htmlex amd64 0:0.0.18.8.rc1.master.bionic.20200630-0 [4,562 kB]
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: u

In [ ]:
import subprocess

def convert_pdf_to_html(pdf_path, html_path):
    command = f"pdf2htmlEX {pdf_path} --dest-dir {html_path}"
    subprocess.call(command, shell=True)

input_pdf = "quarterly-nvidia.pdf"
output_pdf = "quarterly-nvidia"

convert_pdf_to_html(input_pdf, output_pdf)

In [ ]:
from llama_index.readers.file.flat_reader import FlatReader
from pathlib import Path

reader = FlatReader()
qr_2023 = reader.load_data(Path("quarterly-nvidia/quarterly-nvidia.html"))

In [ ]:
from llama_index.node_parser import (
    UnstructuredElementNodeParser,
)

node_parser = UnstructuredElementNodeParser()

In [ ]:
import os
import pickle

if not os.path.exists("qr_2023_nodes.pkl"):
    qr_2023_raw_nodes = node_parser.get_nodes_from_documents(qr_2023)
    pickle.dump(qr_2023_raw_nodes, open("qr_2023_nodes.pkl", "wb"))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
0it [00:00, ?it/s]


In [ ]:
base_nodes_qr_2023, node_mappings_qr_2023 = node_parser.get_base_nodes_and_mappings(
    qr_2023_raw_nodes
)

In [ ]:
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index import VectorStoreIndex

vector_index = VectorStoreIndex(base_nodes_qr_2023)
vector_retriever = vector_index.as_retriever(similarity_top_k=3)
vector_query_engine = vector_index.as_query_engine(similarity_top_k=3)

In [ ]:
from llama_index.retrievers import RecursiveRetriever

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    node_dict=node_mappings_qr_2023,
)
query_engine = RetrieverQueryEngine.from_args(recursive_retriever)

In [ ]:
response = query_engine.query("What was the revenue for Gaming?")
print(str(response))

The revenue for Gaming in the third quarter of fiscal year 2024 was $2.86 billion.


In [ ]:
response = query_engine.query("What about datacenter revenue?")
print(str(response))

Data Center revenue drove roughly half of the overall revenue for the company. It experienced significant growth, with a 279% increase compared to the previous year and a 41% increase compared to the previous quarter. This growth was primarily attributed to improved product mix and strong demand for Data Center products and services.


In [ ]:
response = query_engine.query("What were the third quarter Research and development expenses?")
print(str(response))

The third quarter Research and development expenses were $2,294 million.


In [ ]:
response = query_engine.query("Unamortized debt discount and issuance costs?")
print(str(response))

The context information does not provide any specific details about unamortized debt discount and issuance costs. Therefore, it is not possible to provide an answer to the query based on the given context.
